In [4]:
!pip install mysql-connector-python

In [5]:
! pip install google-cloud-storage

In [9]:
import os
import time
import subprocess
from google.cloud import storage
import getpass

In [19]:
# Set your MySQL database configurations

username = getpass.getpass('Enter username:')
password = getpass.getpass('Enter password:')
public_ip_address = '35.239.201.185'
db_name = 'gmb'
test_db_name = 'gmb_test'

Enter username:········
Enter password:········


In [20]:
# Set your GCS bucket configurations
gcs_bucket_name = 'gmb-database-backup'
gcs_backup_blob_name = 'backup.sql'
# Path to store the temporary backup file
TEMP_BACKUP_PATH = '/tmp/backup.sql'


In [21]:
## create a backup of the database in the GCP S3 folder
def create_mysql_backup():
    try:
        # Construct the mysqldump command
        cmd = f"mysqldump -h {public_ip_address} -u {username} -p{password} {db_name} > {TEMP_BACKUP_PATH}"
        
        # Execute the mysqldump command
        subprocess.run(cmd, shell=True, check=True)
        print("MySQL backup created successfully")
    except Exception as e:
        print("Error creating MySQL backup:", str(e))
    
def upload_backup_to_gcs():
    try:
        # Initialize GCS client
        storage_client = storage.Client()
        bucket = storage_client.bucket(gcs_bucket_name)
        
        # Upload the backup file to GCS
        blob = bucket.blob(gcs_backup_blob_name)
        blob.upload_from_filename(TEMP_BACKUP_PATH)
        print("Backup uploaded to GCS successfully")
    except Exception as e:
        print("Error uploading backup to GCS:", str(e))
        

In [22]:
## Allow user to create a new (test) data base  
def create_test_database():
    try:
        connection = mysql.connector.connect(
            host=public_ip_address,
            user=username,
            password=password,
            database='',
        )       
        cursor = connection.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {test_db_name}")
        cursor.close()
        connection.close()
        print(f"Test database '{test_db_name}' created successfully")
    except Exception as e:
        print("Error creating test database:", str(e))
        

In [23]:
## Test backend code config with new restored database for testing
def test_backend_code():
    try:
        # Initialize MySQL connection
        connection = mysql.connector.connect(
            host=public_ip_address,
            user=username,
            password=password,
            database=db_name  # Use the restored database
        )

        cursor = connection.cursor()

        # Perform actions with the database
        # For example:
        # cursor.execute("SELECT * FROM your_table;")
        # data = cursor.fetchall()
        # print(data)

        cursor.close()
        connection.close()

    except Exception as e:
        print("Error testing backend code:", str(e))
